# Fase 6
## Estrategia detallada paso a paso (qué hacer antes de entrenar)

### 2.1 Pre-filtrado estructural 
- Eliminar columnas con >30–40% de valores nulos (NaN) o imputar si son relevantes.  
- Eliminar columnas con varianza casi nula (constantes).  
- Detectar y eliminar duplicados exactos o altamente correlacionados.  
- Convertir precios a **retornos logarítmicos** y mantener precios si aportan contexto adicional.

---

### 2.2 Priorización por dominio
Seleccionar manualmente variables relevantes por grupo:

- **Acciones bancarias:** precios, volumen, retornos, volatilidades móviles (30d).  
- **Índices:** IBEX, S&P500, DAX — retornos + volatilidades móviles.  
- **Divisas (FX):** EURUSD, EURBRL — retornos o niveles según correlación.  
- **Commodities:** petróleo, oro — retornos o niveles según relación con el mercado.  
- **Macroeconómicos:** PIB, IPC, tipos BCE — incluir rezagos (lags) temporales.  
- **Eventos:** impacto cuantificado (`Impact_Score` o similar).

👉 Idea: mantener **3–8 variables por grupo** para conservar representatividad sin sobrecargar el modelo.

---

### 2.3 Selección estadística automática (complementaria)
Aplicar métodos automáticos sobre el conjunto de entrenamiento:

- **Correlación con el target:** `corrwith(BBVA_Close)` o `corrwith(SANT_Close)` sobre retornos; mantener top N (p.ej. 50).  
- **VarianceThreshold:** eliminar columnas con varianza baja.  
- **Multicolinealidad:** eliminar features con alto VIF (Variance Inflation Factor).  
- **Importancia (árboles):** usar RandomForest o GradientBoost rápido para priorizar las más influyentes.

⚠️ Importante: **filtrar siempre usando solo el conjunto de entrenamiento**, nunca con test.

---

### 2.4 Reducción de dimensionalidad
Opciones recomendadas:

- **PCA:** escalar datos y elegir componentes que expliquen 80–95% de la varianza (≈10–50 componentes).  
- **Autoencoder:** alternativa no lineal que puede capturar relaciones complejas.  
- **Agregación / clustering:** promediar o sintetizar variables muy similares.  
- **Híbrido:** combinar top features seleccionadas + PCA sobre el resto.

---

### 2.5 Recomendaciones numéricas
- Número final de variables:  
  - Interpretables: **30–60 features**.  
  - Compactas (post-PCA/AE): **10–30 componentes**.  
- Si el modelo predice múltiples outputs (BBVA, SANT), mantener suficiente diversidad (≈50 features).  
- Ajustar número según longitud de ventana temporal (60–120 días → más features = más memoria).

---

### 2.6 Otros puntos clave
- **Imputación:** aplicar antes de PCA; `forward-fill` o medianas para macro.  
- **Escalado:** `StandardScaler` o `RobustScaler`, ajustado solo con train.  
- **Lags:** crear lags relevantes (1, 3, 7, 30 días) para macro y FX.  
- **Pruebas controladas:**  
  - Baseline (pocas features manuales).  
  - Variante top-50.  
  - Variante PCA-20.  
  - Comparar rendimiento y coste.

---

### Checklist rápido (FASE 6)
- [ ] Eliminar columnas con >30% NaN  
- [ ] Eliminar columnas con varianza nula  
- [ ] Calcular retornos y volatilidades móviles  
- [ ] Seleccionar top 50 correlaciones  
- [ ] Aplicar PCA/AE a resto y fijar número de componentes  
- [ ] Guardar lista final de variables (`final_features_list.csv`)  
- [ ] Documentar transformaciones (scalers, imputers, PCA)


---
---
---
## Resumen del flujo recomendado antes del modelado RNN

### 1. Limpieza estructural
- Eliminar columnas con más de 30–40% de valores nulos.  
- Eliminar columnas constantes o duplicadas.  
- Imputar los valores faltantes restantes (`forward-fill`, media o mediana según el caso).  
- Escalar todas las variables (`StandardScaler` o `RobustScaler`).

---

### 2. Transformaciones de mercado
- Convertir precios a **retornos logarítmicos**:  
  \[
  r_t = \log\left(\frac{P_t}{P_{t-1}}\right)
  \]
- Calcular **volatilidades móviles** (rolling std) de 30 y 90 días para capturar dinámica temporal.

---

### 3. Selección de variables relevantes
- Calcular la **correlación en valor absoluto** de todas las variables con los **retornos de BBVA y Santander**.  
- Seleccionar las **Top N** (por ejemplo 50–80) variables más correlacionadas.  
- Este subconjunto representa las features más informativas para el modelo.

---

### 4. Reducción de dimensionalidad (PCA)
- Aplicar **PCA** sobre las variables restantes (no seleccionadas en el paso anterior).  
- Conservar las componentes que expliquen entre **90–95% de la varianza total** (≈10–30 componentes).  
- Combinar:
  - Las variables seleccionadas por correlación.  
  - Las componentes principales del PCA.

📦 **Resultado:**  
Un dataset limpio, compacto y optimizado con unas **~60 variables finales**, que combinan la relevancia estadística (correlación) y la representación comprimida (PCA), ideal para la siguiente fase de modelado RNN.


## Libreries


## 1. Limpieza

In [ ]:
# ============================
# 🧹 FASE 6.1 — LIMPIEZA INICIAL (optimizada con NumPy)
# ============================

import pandas as pd
import numpy as np
from pathlib import Path

# ============================
# CONFIG
# ============================
DATA_PATH = Path("../../data/processed/data.csv.gz")

# ============================
# CARGA
# ============================

data = pd.read_csv(
    DATA_PATH,
    compression="gzip",
    parse_dates=["Date"],
    index_col="Date",
    dtype=float  # convierte todas las columnas a float directo
)
print(f"Dataset cargado con {data.shape[0]} filas y {data.shape[1]} columnas.")
display(data.head())

# ============================
# 1️⃣ Conversión de tipos
# ============================
for col in data.select_dtypes(include='object').columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [ ]:
# ============================
# 2️⃣ Eliminación de columnas duplicadas exactas
# ============================
data = data.loc[:, ~data.columns.duplicated()]

In [ ]:
# ============================
# 3️⃣ Eliminación de columnas altamente correlacionadas (>0.98) usando NumPy
# ============================
# Convertir a numpy array
arr = data.to_numpy()
# Calcular correlación
corr_matrix = np.corrcoef(arr, rowvar=False)
# Mantener la matriz superior triangular
upper_tri = np.triu(corr_matrix, k=1)

# Columnas a eliminar
to_drop = [data.columns[i] for i in range(len(data.columns)) if any(upper_tri[:, i] > 0.98)]
print(f"Columnas eliminadas por alta correlación: {len(to_drop)}")

# Eliminar columnas
data = data.drop(columns=to_drop)

In [ ]:
# ============================
# 4️⃣ Comprobaciones finales
# ============================
print(f"Dataset limpio → {data.shape[0]} filas y {data.shape[1]} columnas.")
print(f"Nulos restantes: {data.isna().sum().sum()}")
display(data.head())

In [ ]:
# ============================
# 5️⃣ Guardar dataset limpio temporal
# ============================
output_path = Path("../../data/processed/data_clean.csv.gz")
data.to_csv(output_path, compression="gzip")
print(f"✅ Dataset limpio guardado en: {output_path}")